In [ ]:
!pip install -U albumentations ultralytics optuna huggingface_hub
!git clone https://github.com/sathishkumar67/ADIS.git
!mv /teamspace/studios/this_studio/ADIS/* /teamspace/studios/this_studio/

In [2]:
# necessary imports
from __future__ import annotations
import os
import joblib
import optuna
from huggingface_hub import hf_hub_download
from utils import unzip_file
from model import YOLO11Model

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/teamspace/studios/this_studio/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
# Define the global variables
REPO_ID = "pt-sk/ADIS" 
FILENAME_IN_REPO = "dataset.zip"
LOCAL_DIR = os.getcwd()
TRAIN_PATH = f"{LOCAL_DIR}/dataset/train"
VAL_PATH = f"{LOCAL_DIR}/dataset/val"
TEST_PATH = f"{LOCAL_DIR}/dataset/test"
DATASET_PATH = f"{LOCAL_DIR}/{FILENAME_IN_REPO}"
REPO_TYPE = "dataset"
NUM_CLASSES = 10                                               
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', "Squirrel", 'Eagle', 'Goat', 'Rodents', 'Snake'] 
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"
MODEL_PATH = "yolo11s.pt"

In [4]:
# download the dataset and unzip it
hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)
unzip_file(DATASET_PATH, LOCAL_DIR)


# Get the number of CPU cores
num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

dataset.zip:   0%|          | 0.00/9.57G [00:00<?, ?B/s]

Could not set the permissions on the file '/teamspace/studios/this_studio/.cache/huggingface/download/v7Xt8H1GQ8W09J-6CSgzA0zuBH0=.d957c623096e6fa45d2927706b5d650c880bbad6352ec28015eb8af4f6b5777f.incomplete'. Error: [Errno 13] Permission denied: '/teamspace/studios/tmp_33d94f6b-a7b2-403e-8a0f-8199328bf105'.
Continuing without setting permissions.
Unzipping: 100%|██████████| 9.60G/9.60G [00:55<00:00, 173MB/s]

Number of CPU cores: 16


In [7]:
# split paths for model
data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

# write data yaml file
with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)
    print("data yaml file written!.............")

data yaml file written!.............


In [8]:
# Define the objective function
def objective(trial):
    
    # Define callback to report intermediate results
    def on_train_epoch_end(score, epoch):
        trial.report(score, step=epoch)  
        if trial.should_prune():
            raise optuna.TrialPruned()

    callbacks = {
        "on_train_epoch_end" : on_train_epoch_end
    }
    
    # Define hyperparameters using Optuna suggestions
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-3, log=True)
    lrf = trial.suggest_float("lrf", 0.1, 1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.00001, 0.01, log=True)
    warmup_momentum = trial.suggest_float("warmup_momentum", 0.6, 0.9)
    momentum = trial.suggest_float("momentum", 0.8, 0.99)
    
    CONFIG_DICT = {
    "task": "detect",
    "mode": "train",
    "bohb": True,
    "custom_callbacks": callbacks,
    "data": DATA_YAML_FILE,
    "batch": 640,
    "imgsz": 320,
    "save": True,
    "device": 0,
    "workers": num_cores,
    "pretrained": True,
    "optimizer": "AdamW",
    "seed": 42,
    "epochs": 5,
    "warmup_epochs": 2,
    "patience": 1}

    # Train YOLO model
    model = YOLO11Model(MODEL_PATH)
    model.train(**CONFIG_DICT, lr0=lr0, lrf=lrf, momentum=momentum, weight_decay=weight_decay, warmup_momentum=warmup_momentum)
    
    # Return validation mAP as the objective value
    return model.score 

In [ ]:
# Define the study
NUM_TRIALS = 90
study = optuna.create_study(direction='maximize', 
                            sampler=optuna.samplers.TPESampler(), 
                            pruner=optuna.pruners.HyperbandPruner(),
                            study_name="yolo11_tuning",
                            load_if_exists=True)

# Optimize with a callback to stop after NUM_TRIALS complete trials
study.optimize(
    objective,
    n_trials=NUM_TRIALS)

In [21]:
joblib.dump(study, "optuna_study2.pkl")

['optuna_study2.pkl']

In [1]:
import joblib

In [5]:
study = joblib.load("optuna_study_3.pkl")

In [6]:
# convert the study results to a dataframe
df = study.trials_dataframe()
df

,number,value,datetime_start,datetime_complete,duration,params_lr0,params_lrf,params_momentum,params_warmup_momentum,params_weight_decay,system_attrs_completed_rung_0,system_attrs_completed_rung_1,state
0,0,0.4430,2025-03-28 06:15:27.816426,2025-03-28 06:19:04.140992,0 days 00:03:36.324566,0.000011,0.466062,0.817348,0.638537,0.000032,NaN,NaN,COMPLETE
1,1,0.5372,2025-03-28 06:19:04.142210,2025-03-28 06:22:16.613657,0 days 00:03:12.471447,0.000033,0.300500,0.887356,0.684525,0.002748,0.3883,NaN,COMPLETE
2,2,0.5666,2025-03-28 06:22:16.615294,2025-03-28 06:25:28.934572,0 days 00:03:12.319278,0.000043,0.392015,0.811246,0.889170,0.000014,0.4057,NaN,COMPLETE
3,3,0.3049,2025-03-28 06:25:28.938154,2025-03-28 06:27:30.420854,0 days 00:02:01.482700,0.000014,0.184120,0.952710,0.883140,0.000225,0.3049,NaN,PRUNED
4,4,0.5742,2025-03-28 06:27:30.421519,2025-03-28 06:30:44.269308,0 days 00:03:13.847789,0.000149,0.132252,0.981286,0.680623,0.001565,0.0277,0.4167,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,0.4814,2025-03-28 09:35:06.713952,2025-03-28 09:37:11.033539,0 days 00:02:04.319587,0.000387,0.105538,0.825443,0.730479,0.000050,0.4814,NaN,PRUNED
86,86,0.4448,2025-03-28 09:37:11.034218,2025-03-28 09:39:16.916572,0 days 00:02:05.882354,0.000530,0.100153,0.820707,0.711169,0.000068,0.4448,NaN,PRUNED
87,87,0.4955,2025-03-28 09:39:16.917411,2025-03-28 09:41:25.482915,0 days 00:02:08.565504,0.000199,0.139060,0.805818,0.724256,0.000030,0.4955,NaN,PRUNED
88,88,0.0598,2025-03-28 09:41:25.483616,2025-03-28 09:42:30.112375,0 days 00:01:04.628759,0.000263,0.132376,0.909658,0.856677,0.000045,0.0598,NaN,PRUNED
